# TODO:
### update frsq poi mapping
### assigning rltn tag to equal ln may not be a good idea. 

for exmaple, when the role of a way is inner(http://www.openstreetmap.org/way/48076826), meaning the the way serve a different function than rltn(http://www.openstreetmap.org/relation/381363)

it seems that "inner" role is the only fallback

examples:
- set(['cycling facilities', 'retail shop']) http://www.openstreetmap.org/way/48076826 set(['transportation']) http://www.openstreetmap.org/relation/381363
- set(['transportation']) http://www.openstreetmap.org/way/48081175 set(['outdoors and recreation']) http://www.openstreetmap.org/relation/3213622
- set(['professional service']) http://www.openstreetmap.org/way/48085321 set(['transportation']) http://www.openstreetmap.org/relation/914244
- set(['cycling facilities']) http://www.openstreetmap.org/way/48168497 set(['transportation']) http://www.openstreetmap.org/relation/383188
- set(['cycling facilities']) http://www.openstreetmap.org/way/52303179 set(['transportation']) http://www.openstreetmap.org/relation/450333
- set(['professional service']) http://www.openstreetmap.org/way/66417858 set(['outdoors and recreation']) http://www.openstreetmap.org/relation/3213622
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/173918307 set(['cycling facilities']) http://www.openstreetmap.org/relation/2322204
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/66418767 set(['cycling facilities']) http://www.openstreetmap.org/relation/2322204
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/237887425 set(['cycling facilities']) http://www.openstreetmap.org/relation/3205766
- set(['outdoors and recreation']) http://www.openstreetmap.org/way/237887426 set(['cycling facilities']) http://www.openstreetmap.org/relation/3205766
- set(['cycling facilities']) http://www.openstreetmap.org/way/203025090 set(['transportation']) http://www.openstreetmap.org/relation/2724481
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170966
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170968
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579359 set(['transportation']) http://www.openstreetmap.org/relation/5170967
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170968
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170966
- set(['cycling facilities']) http://www.openstreetmap.org/way/288579360 set(['transportation']) http://www.openstreetmap.org/relation/5170967
- set(['cycling facilities']) http://www.openstreetmap.org/way/296755159 set(['transportation']) http://www.openstreetmap.org/relation/2723927
- set(['cycling facilities']) http://www.openstreetmap.org/way/296794684 set(['transportation']) http://www.openstreetmap.org/relation/2723927

# function of get_feature_poi

1. [x] mapped frsq venues to poi categories --> filter frsq venues by obj_near_segment
2. [x] mapped osm data to poi categories, filter osm data by obj_near_segment
3. [x] remove overlap between osm and frsq
4. [x] Visalization:
    - [x] poi categories distribution
        1. [x] frsq venues near segments
        2. [x] osm venues near segments
        3. [x] final poi near segments(after removing overlap)
    - [x] poi categories per segment distribution

In [19]:
import src.constants; reload(src.constants)
import src.geom_helper; reload(src.geom_helper)
import src.ftr_poi; reload(src.ftr_poi)
import src.utils; reload(src.utils)
from src.constants import fn_frsq_venues_dc, fn_segments_dc, epsg_dc, fn_osm_db_dc, fn_poi_frsq_dc, fn_poi_osm_dc
from src.constants import fn_poi_distr_dc, fn_feature_poi_dc, fn_poi_seg_cvrg_dc, fn_poi_boxplot_per_seg_dc
from src.constants import fn_mapping_for_fs, fn_mapping_for_osm, poi_categories
from src.geom_helper import bfr_20m, objs_near_segs, objs_near_segs_store, gpdf_equal

import geopandas as gp
import pandas as pd
import numpy as np

from src.ftr_poi import *

In [20]:
data_dir = 'data/'
path_frsq_venues_dc = data_dir + fn_frsq_venues_dc
path_osm_db_dc = data_dir + fn_osm_db_dc
path_segs_dc = data_dir + fn_segments_dc
path_mapping_for_fs = data_dir+fn_mapping_for_fs
path_mapping_for_osm = data_dir+fn_mapping_for_osm
path_poi_distr_dc = data_dir + fn_poi_distr_dc
path_feature_poi_dc = data_dir + fn_feature_poi_dc

path_poi_seg_cvrg_dc = data_dir + fn_poi_seg_cvrg_dc
path_poi_boxplot_per_seg_dc = data_dir + fn_poi_boxplot_per_seg_dc

In [23]:
seg_poi_features_dc, poi_distr_dc, poi_near_segs_dc, seg_poi_index_dc = get_feature_poi(
    path_frsq_venues_dc, path_osm_db_dc, path_segs_dc, path_mapping_for_fs, path_mapping_for_osm, path_poi_distr_dc, 
    path_feature_poi_dc, path_poi_seg_cvrg_dc, path_poi_boxplot_per_seg_dc,
    bfr_20m, 5, bfr_crs=epsg_dc, init_crs=4326, debug=True)

===========mapping  frsq venues to poi categories===========
# venues with poi = 28740
venues without poi category: #venues=1447, #frsq_categories=1
top ten unmapped frsq_categories {'no category': 1447}
===========mapping osm to poi category===========
# rows in table tag  = 1484405
# rows after mapping = 44060
# objs have category = 32808
# objs have category by ot: {'R': 1079, 'W': 20010, 'N': 11719}
# categories: # objs = {1: 32183, 2: 620, 3: 5}


d:\program files\anaconda2\lib\site-packages\pandas\core\generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


=====remove_osm_frsq_overlap=====
# conn components(osm_pts connected with frsq):  2658
# pairs of overlap (881, 13)
# pairs by overlap criteria {'same category': 111, 'similar name': 770}
distribution of # pairs of overlap within each connected component {1: 679, 2: 66, 3: 12, 4: 6, 5: 2}
# overlap points in osm = 849, # points in fs = 863
# independent points in osm = 10914, # points in fs = 27877
# total points in osm = 11725, # points in fs = 28740
# venues in city = 28740
# osm after clean in city, # points = 11725, # line/polygon = 23373
# poi in dc = 63045, # poi near seg dc = 43680
# objs have category near seg by ot: {'R': 1398, 'N_fq': 721, 'W': 14389, 'N': 8086}
plot poi distribution
# segs with poi: 12514/13522=92.5454814377%
plot poi per seg


In [4]:
seg_poi_features_dc, poi_distr_dc, poi_near_segs_dc, seg_poi_index_dc = get_feature_poi(
    path_frsq_venues_dc, path_osm_db_dc, path_segs_dc, path_mapping_for_fs, path_mapping_for_osm, path_poi_distr_dc, 
    path_feature_poi_dc, path_poi_seg_cvrg_dc, path_poi_boxplot_per_seg_dc,
    bfr_20m, 5, bfr_crs=epsg_dc, init_crs=4326, debug=True)

===========mapping  frsq venues to poi categories===========
# venues with poi = 27048
venues without poi category: #venues=3139, #frsq_categories=63
top ten unmapped frsq_categories {u'': 1447, u'Building': 882, u'TV Station': 11, u'Exhibit': 51, u'Moving Target': 121, u'Zoo': 14, u'Festival': 6, u'Conference': 7, u'Intersection': 55, u'Road': 450}
===========mapping osm to poi category===========
# rows in table tag  = 1484405
# rows after mapping = 44060
# objs have category = 32808
# objs have category by ot: {'R': 1079, 'W': 20010, 'N': 11719}
# categories: # objs = {1: 32183, 2: 620, 3: 5}


d:\program files\anaconda2\lib\site-packages\pandas\core\generic.py:2177: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


=====remove_osm_frsq_overlap=====
# conn components(osm_pts connected with frsq):  2727
# pairs of overlap (891, 13)
# pairs by overlap criteria {'same category': 111, 'similar name': 780}
distribution of # pairs of overlap within each connected component {1: 681, 2: 70, 3: 12, 4: 6, 5: 2}
# overlap points in osm = 857, # points in fs = 873
# independent points in osm = 10906, # points in fs = 29314
# total points in osm = 11725, # points in fs = 30187
# venues in city = 30187
# osm after clean in city, # points = 11725, # line/polygon = 23373
# poi in dc = 64484, # poi near seg dc = 44668
# objs have category near seg by ot: {'R': 1398, 'N_fq': 729, 'W': 14389, 'N': 8081}
plot poi distribution
# segs with poi: 12530/13522=92.6638071291%
plot poi per seg
